In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
from tqdm import tqdm

### Gets all the words for search from CSV file 

In [2]:
def get_words(file_add):
    word_csv = pd.read_csv(file_add)
    words = []
    for i in word_csv['word']:
        words.append(i)
    return words    

Returns a list containg all the words from CSV file 

### Extracts the meanings of a single word from Google Dictionary

 Google Dictionary was an online dictionary service, originating in its Google Translate service.
 Link : https://googledictionary.freecollocation.com/

In [3]:
def get_meaning(word):
    url = "https://googledictionary.freecollocation.com/meaning?word=" + word
    response = requests.get(url)
    soupBody = BeautifulSoup(response.content,features = "html.parser")
    text = soupBody.findAll("li", {"style" : "list-style:decimal"})
    meanings_with_codes = []
    for i in text: 
        meanings_with_codes.append(i.text)
    meanings_raw = []
    for i in range(0, len(meanings_with_codes)):
        if "-" in meanings_with_codes[i]:
            meanings_raw.append(meanings_with_codes[i][:meanings_with_codes[i].index("-")])
        else :
            meanings_raw.append(meanings_with_codes[i])
    meanings_1 = []
    for sub in meanings_raw:
        meanings_1.append(sub.replace("\n", ""))
    meanings = []
    for sub in meanings_1:
        meanings.append(sub.replace(";", " or"))     
    return meanings  

Returns a list containg all the meanings of word 

### Uses  get_meaning() function to extract the meanings of all the words

In [4]:
def meaning_extraction(words):
    rows = []
    for i in tqdm(range(len(words)), desc = "Meaning Extraction"):
        meanings = get_meaning(words[i])
        x = []
        for j in (range(len(meanings))):
            x.append(meanings[j])
        rows.append([words[i]]+x)
    return rows    

Returns a list rows containg all the meanings of all word in a 2D list format 

### Writes the meanings in CSV file  

In [5]:
def write_csv(filename,header,rows):
    with open("results/"+filename, 'w') as csvfile:  
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerow(header)  
        csvwriter.writerows(rows)
    return None

### Main function 

In [6]:
def main(file_add,filename):
    words = get_words(file_add)
    rows = meaning_extraction(words)
    
    header = ["word","Meaning 1","Meaning 2", "Meaning 3", "Meaning 4"]
    write_csv(filename,header,rows)
    print("File Saved Successfully")
    return None

### Defining the  file address and name

In [7]:
file_add = "data/words.csv"
filename = "results.csv"

### Calling the Main function  

In [12]:
main(file_add,filename)

Meaning Extraction: 100%|██████████| 487/487 [05:28<00:00,  1.48it/s]

File Saved Successfully
